In [17]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID
from jose import jws

## Helper function

In [18]:
secrets_resolver = SecretsResolverDemo()

In [19]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [20]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Reading issuer OOB message

In [37]:
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiNzk0Mjc4MzctY2MwNi00ODUzLWJiMzktNjg2ZWFjM2U2YjlhIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNtczU1NVloRnRobjFXVjhjaURCcFptODZoSzl0cDgzV29qSlVteFBHazFoWi5WejZNa21kQmpNeUI0VFM1VWJiUXc1NHN6bTh5dk1NZjFmdEdWMnNRVllBeGFlV2hFLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwibGFiZWwiOiJNZWRpYXRvciIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ


In [38]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '79427837-cc06-4853-bb39-686eac3e6b9a', 'from': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'label': 'Mediator', 'accept': ['didcomm/v2']}}


## Prepare request-credential

The holder needs to create a did:peer to communicate with the issuer:

In [39]:
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LScQkDHALNyHrb1tcmwG6ykqqpK1S3JKzD1ZSu6yH86fdH.Vz6MkjZXCxBuNF2zvcBvyGAbKo2HEimZRDtehLe7eaREnppL2.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


The following is a JSON-LD credential request that the holder will submmit to the issuer.

In [40]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-vc-edu-plugfest-1-context.json"
        ],
        "id": str(uuid.uuid4()),
        "type": ["VerifiableCredential", "OpenBadgeCredential"],
        "issuer": "TBD",
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
          "credentialSubject": {
            "type": "AchievementSubject",
            "id": holder_did,
            "achievement": {
              "type": "Achievement",
              "name": "IIW 2022 DEMO",
              "description": "This wallet can display this Open Badge 3.0",
              "criteria": {
                "type": "Criteria",
                "narrative": "The first cohort of the JFF Plugfest 1 in May/June of 2021 collaborated to push interoperability of VCs in education forward."
              },
              "image": "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/images/plugfest-1-badge-image.png"
            }
          },
        "options": {
            "proofType": "Ed25519Signature2018"
        }
    }
}

In [25]:
sign = jws.sign(credential_request, 'secretasasdsadasd', algorithm='HS256')
print(sign)

eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjcmVkZW50aWFsIjp7IkBjb250ZXh0IjpbImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIiwiaHR0cHM6Ly93M2MtY2NnLmdpdGh1Yi5pby92Yy1lZC9wbHVnZmVzdC0xLTIwMjIvamZmLXZjLWVkdS1wbHVnZmVzdC0xLWNvbnRleHQuanNvbiJdLCJpZCI6ImE2YWFjZmViLThiZDMtNGJhNy1hYTg1LTQ3NWMzZTUwNzIyNCIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJPcGVuQmFkZ2VDcmVkZW50aWFsIl0sImlzc3VlciI6IlRCRCIsImlzc3VhbmNlRGF0ZSI6IjIwMjItMTAtMjZUMTY6MTY6MDRaIiwiY3JlZGVudGlhbFN1YmplY3QiOnsidHlwZSI6IkFjaGlldmVtZW50U3ViamVjdCIsImlkIjoiZGlkOnBlZXI6Mi5FejZMU3J0UUFrYzJhQ3ZoR1RwQ01tRDlYSmRBS0M2R2N6c1lmYUNrMzF0NFAyanZqLlZ6Nk1raHNHYUE0dENNRlJ1bjlZTFpRMktiaWF5ZnVMUFRhcGVqWVJmalVwTnFGUmMuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5b2IyeGtaWElpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIiwiYWNoaWV2ZW1lbnQiOnsidHlwZSI6IkFjaGlldmVtZW50IiwibmFtZSI6IklJVyAyMDIyIERFTU8iLCJkZXNjcmlwdGlvbiI6IlRoaXMgd2FsbGV0IGNhbiBkaXNwbGF5IHRoaXMgT3BlbiBCYWRnZSAzLjAiLCJjcml0ZXJpYSI6eyJ0eXBlIjoiQ3J

Finally the holder prepares the didcomm message following issue-credential/3.0 protocol:

In [41]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=credential_request)
                )
    ]
                        
)

In [42]:
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

From the issuer DID obtained in the OOB QR Code, we can get the DID document:

In [45]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))

And from there get the issuer's endpoint

In [46]:
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)

https://mediator.rootsid.cloud


Finally sending the request to the issuer:

In [47]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, json = json.loads(holder_packed_msg.packed_msg))
print(resp)


<Response [202]>


### Issued Verifiable Credential Received

In [48]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [49]:
print(credential_unpack_msg)

UnpackResult(message=Message(id='21499f35-8d86-42ee-802e-2508f2abfc6b', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={'goal_code': 'issue-credential', 'comment': 'some comment'}, frm=None, to=None, created_time=None, expires_time=None, from_prior=FromPrior(iss='did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', sub='did:peer:2.Ez6LSjcRc668ybsiz4EeEEAgJCCHi1vifVwwzhbyFRPA3Jhpe.Vz6Mkvd8ExPq7oQao1ZyneZ5aB8r8duarYXcQcW1XaRiJszZC.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', aud=None, exp=None, nbf=None, iat=None, jti=None), please_ack=None, ack=None, thid=None, pthid=None, attachments=[Attachment(data=AttachmentDataJson(json={'@context': ['https://www.w3.org/2018/credentials/v1', 'https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-

In [50]:
print(json.dumps(credential_unpack_msg.message.attachments[0].data.json, indent=4))

{
    "@context": [
        "https://www.w3.org/2018/credentials/v1",
        "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-vc-edu-plugfest-1-context.json"
    ],
    "id": "e8146df1-f3ec-4f8e-aa0d-af1cf89b4c26",
    "type": [
        "VerifiableCredential",
        "OpenBadgeCredential"
    ],
    "issuer": {
        "type": "Profile",
        "id": "did:peer:2.Ez6LSjcRc668ybsiz4EeEEAgJCCHi1vifVwwzhbyFRPA3Jhpe.Vz6Mkvd8ExPq7oQao1ZyneZ5aB8r8duarYXcQcW1XaRiJszZC.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0",
        "name": "IIW 2022",
        "url": "https://www.jff.org/",
        "image": "https://kayaelle.github.io/vc-ed/plugfest-1-2022/images/JFF_LogoLockup.png"
    },
    "issuanceDate": "2022-10-27T13:43:10Z",
    "credentialSubject": {
        "type": "AchievementSubject",
        "id": "did:peer:2.Ez6LScQkDHALNyHrb1tcmwG6ykqqpK1S3JKzD1ZSu6yH86fdH.Vz6MkjZXCxBuNF2zvcBvyGAbKo2HEimZRDtehLe7eaREnppL2.SeyJpZCI6Im5ld